In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [48]:
# input files
var_qc_f    <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/annotation_20201012/ukb24983_cal_hla_cnv.var_QC.tsv.gz'
cal_annot_f <- '/oak/stanford/groups/mrivas/ukbb24983/cal/annotation_20201002/ukb24983_cal_cALL_v2_hg19.vep101-loftee.Csq.tsv.gz'
cnv_annot_f <- '/oak/stanford/groups/mrivas/ukbb24983/cnv/annotation_20201003/cnv.vep101-loftee.20201009.Csq.tsv.gz'
ld_indep_f  <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/ld_indep_20201015/ukb24983_cal_hla_cnv.white_british.bool.prune.in.tsv.gz'

# output
var_annot_f <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/annotation_20201012/ukb24983_cal_hla_cnv.annot_20201023.tsv'
var_annot_compact_f <- '/oak/stanford/groups/mrivas/ukbb24983/array-combined/annotation/annotation_20201012/ukb24983_cal_hla_cnv.annot_compact_20201023.tsv'


In [2]:
cat_or_zcat <- function(f){
    ifelse(endsWith(f, '.zst'), 'zstdcat', ifelse(endsWith(f, '.gz'), 'zcat', 'cat'))
}

fread_CHROM <- function(f){
    fread(cmd=paste(cat_or_zcat(f), f), colClasses = c('#CHROM'='character')) %>% rename('CHROM'='#CHROM')
}


In [93]:
get_POS_total <- function(df){
    # compute the location on the linear coordinate system (chr1-22, X, Y, MT) for plotting
    df %>% select(CHROM, POS, ID) %>%
    mutate(CHROM_X = if_else(CHROM == 'XY', 'X', CHROM)) -> CHROM_POS_df

    CHROM_POS_df %>% group_by(CHROM_X) %>%
    summarise(chr_len = max(POS), .groups = 'drop') %>%
    left_join(data.frame(CHROM_X = c(1:22, 'X', 'Y', 'MT'), CHROM_order=1:25, stringsAsFactors=F), by='CHROM_X') %>%
    arrange(CHROM_order) %>% mutate(CHROM_tot= cumsum(as.numeric(chr_len)) - chr_len) %>%
    select(CHROM_X, CHROM_tot) %>% left_join(CHROM_POS_df, by='CHROM_X') %>%
    mutate(POS_total = POS + CHROM_tot) %>%
    select(ID, POS_total)    
}


In [20]:
var_qc_f    %>% fread_CHROM() -> var_qc_df
cal_annot_f %>% fread_CHROM() -> cal_annot_df
cnv_annot_f %>% fread_CHROM() -> cnv_annot_df
ld_indep_f  %>% fread(colClasses = c('#ID'='character')) %>% rename('ID'='#ID') -> ld_indep_df


In [30]:
cnv_annot_df %>%
select(-pvar_order) %>%
rename('CNV_POS_s'='POS_s', 'CNV_POS_e'='POS_e') %>%
bind_rows(cal_annot_df) -> cal_cnv_annot_df


In [95]:
cal_cnv_annot_df %>%
select(-CHROM, -POS, -REF, -ALT) %>%
right_join(
    var_qc_df %>%
    mutate(sort_order=1:n()),
    by='ID'
) %>%
mutate(ld_indep = ID %in% (ld_indep_df$ID)) %>%
arrange(sort_order) %>% 
select(-sort_order) %>% 
left_join(get_POS_total(var_qc_df), by='ID') -> full_df


In [96]:
full_df %>% dim()

[1] 1080968     160

In [97]:
cols <- c(
    'CHROM', 'POS', 'ID', 'REF', 'ALT', 
    'FILTER', 'POS_total', 'Allele', 'Csq', 'Consequence', 
    'SYMBOL', 'Gene', 'ld_indep', 'geno_data_source', 'array',
    'CNV_POS_s', 'CNV_POS_e', 'UKB_white_british_MAF', 
    'hwe_p', 'mgi_notes', 'f_miss', 'f_miss_UKBB', 'f_miss_UKBL',
    'LoF', 'LoF_filter', 'LoF_flags', 'LoF_info'
)


In [98]:
full_df %>% 
select(all_of(c(cols, 'HGVSp'))) %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(var_annot_compact_f, sep='\t', na = "NA", quote=F)


In [99]:
full_df %>% 
select(all_of(c(cols, setdiff(colnames(full_df), cols)))) %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(var_annot_f, sep='\t', na = "NA", quote=F)


In [100]:
full_df %>% 
select(all_of(c(cols, setdiff(colnames(full_df), cols)))) %>%
colnames()

[1] "CHROM"                                 
  [2] "POS"                                   
  [3] "ID"                                    
  [4] "REF"                                   
  [5] "ALT"                                   
  [6] "FILTER"                                
  [7] "POS_total"                             
  [8] "Allele"                                
  [9] "Csq"                                   
 [10] "Consequence"                           
 [11] "SYMBOL"                                
 [12] "Gene"                                  
 [13] "ld_indep"                              
 [14] "geno_data_source"                      
 [15] "array"                                 
 [16] "CNV_POS_s"                             
 [17] "CNV_POS_e"                             
 [18] "UKB_white_british_MAF"                 
 [19] "hwe_p"                                 
 [20] "mgi_notes"                             
 [21] "f_miss"                                
 [22] "f_miss_UKBB"                           
 [23] "f_miss_UKBL"                           
 [24] "LoF"                                   
 [25] "LoF_filter"                            
 [26] "LoF_flags"                             
 [27] "LoF_info"                              
 [28] "IMPACT"                                
 [29] "Feature_type"                          
 [30] "Feature"                               
 [31] "BIOTYPE"                               
 [32] "EXON"                                  
 [33] "INTRON"                                
 [34] "HGVSc"                                 
 [35] "HGVSp"                                 
 [36] "cDNA_position"                         
 [37] "CDS_position"                          
 [38] "Protein_position"                      
 [39] "Amino_acids"                           
 [40] "Codons"                                
 [41] "Existing_variation"                    
 [42] "ALLELE_NUM"                            
 [43] "DISTANCE"                              
 [44] "STRAND"                                
 [45] "FLAGS"                                 
 [46] "VARIANT_CLASS"                         
 [47] "SYMBOL_SOURCE"                         
 [48] "HGNC_ID"                               
 [49] "CANONICAL"                             
 [50] "MANE"                                  
 [51] "TSL"                                   
 [52] "APPRIS"                                
 [53] "CCDS"                                  
 [54] "ENSP"                                  
 [55] "SWISSPROT"                             
 [56] "TREMBL"                                
 [57] "UNIPARC"                               
 [58] "GENE_PHENO"                            
 [59] "SIFT"                                  
 [60] "PolyPhen"                              
 [61] "DOMAINS"                               
 [62] "miRNA"                                 
 [63] "HGVS_OFFSET"                           
 [64] "AF"                                    
 [65] "AFR_AF"                                
 [66] "AMR_AF"                                
 [67] "EAS_AF"                                
 [68] "EUR_AF"                                
 [69] "SAS_AF"                                
 [70] "AA_AF"                                 
 [71] "EA_AF"                                 
 [72] "gnomAD_AF"                             
 [73] "gnomAD_AFR_AF"                         
 [74] "gnomAD_AMR_AF"                         
 [75] "gnomAD_ASJ_AF"                         
 [76] "gnomAD_EAS_AF"                         
 [77] "gnomAD_FIN_AF"                         
 [78] "gnomAD_NFE_AF"                         
 [79] "gnomAD_OTH_AF"                         
 [80] "gnomAD_SAS_AF"                         
 [81] "MAX_AF"                                
 [82] "MAX_AF_POPS"                           
 [83] "CLIN_SIG"                              
 [84] "SOMATIC"                               
 [85] "PHENO"                                 
 [86] "